In [1]:
!pip install git+git://github.com/davidbau/quick-netdissect.git#egg=netdissect

  Cloning git://github.com/davidbau/quick-netdissect.git to c:\users\lucy\appdata\local\temp\pip-build-mv64krpi\netdissect
fatal: unable to connect to github.com:
github.com[0: 192.30.255.112]: errno=Invalid argument
github.com[1: 192.30.255.113]: errno=Invalid argument



Command "git clone -q git://github.com/davidbau/quick-netdissect.git C:\Users\Lucy\AppData\Local\Temp\pip-build-mv64krpi\netdissect" failed with error code 128 in None
You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
!pip install torch

You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
<a href="https://colab.research.google.com/github/FrederikWarburg/CS-294-131-Trustworthy-Deep-Learning/blob/master/dissect_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import make_grid
from torch.autograd import Variable
import numpy as np


class Generator(nn.Module):
    def __init__(self, img_size, latent_dim, dim):
        super(Generator, self).__init__()

        self.dim = dim
        self.latent_dim = latent_dim
        self.img_size = img_size
        self.feature_sizes = int(self.img_size[0] / 16), int(self.img_size[1] / 16)

        self.latent_to_features = nn.Sequential(
            nn.Linear(latent_dim, 8 * dim * self.feature_sizes[0] * self.feature_sizes[1]),
            nn.ReLU()
        )

        self.features_to_image = nn.Sequential(
            nn.ConvTranspose2d(8 * dim, 4 * dim, 4, 2, 1),
            nn.ReLU(),
            nn.BatchNorm2d(4 * dim),
            nn.ConvTranspose2d(4 * dim, 2 * dim, 4, 2, 1),
            nn.ReLU(),
            nn.BatchNorm2d(2 * dim),
            nn.ConvTranspose2d(2 * dim, dim, 4, 2, 1),
            nn.ReLU(),
            nn.BatchNorm2d(dim),
            nn.ConvTranspose2d(dim, self.img_size[2], 4, 2, 1),
            nn.Sigmoid()
        )

    def forward(self, input_data):
        # Map latent into appropriate size for transposed convolutions
        x = self.latent_to_features(input_data)
        # Reshape
        x = x.view(-1, 8 * self.dim, self.feature_sizes[0], self.feature_sizes[1])
        # Return generated image
        return self.features_to_image(x)

    def sample_latent(self, num_samples):
        return torch.randn((num_samples, self.latent_dim))

In [4]:

generator = torch.load("gen_mnist_model_epoch_200.pt", map_location='cpu')
print(generator)
# generater.load_state_dict(torch.load(".sample_data/gen_mnist_model_epoch_200.pt", map_location='cpu'))
# generater.eval()


Generator(
  (latent_to_features): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
  )
  (features_to_image): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
    (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ConvTranspose2d(16, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): Sigmoid()
  )
)


In [5]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import h5py

def get_multi_mnist_dataloaders(batch_size=128):
    # Resize images so they are a power of 2
    all_transforms = transforms.Compose([
        transforms.ToPILImage(),
#         transforms.Resize(32),
        transforms.Resize(100),
        transforms.ToTensor()
    ])

    train_data = MultiMNIST('MNIST_synthetic.h5',  train = True, transform = all_transforms)
    test_data = MultiMNIST('MNIST_synthetic.h5', train = False, transform = all_transforms)

    # Create dataloaders
#     train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, num_workers=16)
#     test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16)
#     return train_loader, test_loader
    return train_data, test_data

class MultiMNIST(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, path = 'MNIST_synthetic.h5', train = True, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        super(MultiMNIST, self).__init__()
        self.path = path
        self.transform = transform

        f = h5py.File(self.path, 'r')

        if train:
            self.X = list(f['train_dataset'])
            self.y = list(f['train_labels'])
            self.seg = list(f['train_segmentations'])

        else:
            self.X = list(f['test_dataset'])
            self.y = list(f['test_labels'])
            self.seg = list(f['test_segmentations'])

        if False:
            self.X = list(f['val_dataset'])
            self.y = list(f['val_labels'])
            self.seg = list(f['val_segmentations'])

    def __len__(self):
        return len(self.X)
    
#     def __iter__(self):
#         return DataLoaderIter(self)

    def __getitem__(self, index):#, seg=False):

        img = self.X[index]
        target = self.y[index]
        seg = self.seg[index]

        if self.transform:
            img = self.transform(img)

        return img, target, seg

In [7]:
from netdissect import retain_layers, dissect
from netdissect import ReverseNormalize

generator.eval()
generator.cuda()
  
#   for name, layer in generator.named_modules():
#     print(name)
#============ result: 
# latent_to_features
# latent_to_features.0
# latent_to_features.1
# features_to_image
# features_to_image.0
# features_to_image.1
# features_to_image.2
# features_to_image.3
# features_to_image.4
# features_to_image.5
# features_to_image.6
# features_to_image.7
# features_to_image.8
# features_to_image.9
# features_to_image.10

retain_layers(generator, ['features_to_image.0',
                          'features_to_image.1', 
                          'features_to_image.2', 
                          'features_to_image.3', 
                          'features_to_image.4',
                          'features_to_image.5',
                          'features_to_image.6',
                          'features_to_image.7',
                          'features_to_image.8',
                          'features_to_image.9',
                          'features_to_image.10'])
bds, _ = get_multi_mnist_dataloaders()
dissect('sample_data/', generator, bds,
        recover_image=ReverseNormalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        batch_size=100,
        examples_per_unit=10)


ModuleNotFoundError: No module named 'netdissect'

In [ ]:
 import netdissect

ModuleNotFoundError: No module named 'netdissect'

In [ ]:
import torch